In [ ]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import scipy
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip "/drive/My Drive/kernelGAN_Experiments/codes/KernelGAN_B_and_O_F_hist_dif.zip" -d "/"

In [ ]:
base_dir = "/content/drive/MyDrive/KernelGAN-Masks"
weights_dir = base_dir+"/weights"
img_dir = base_dir+"/imgs"
masks_dir = base_dir+"/masks"
exp_dir = base_dir+"/exps"

# prepare inputs and output dir

In [ ]:
def create_folders(exp_dir):
  files = os.listdir(img_dir)
  for img_name in files:
    print(img_name)
    dir_path = os.path.join(exp_dir, img_name)
    print(dir_path)
    if not os.path.exists(dir_path):
      os.mkdir(dir_path)
      print("create :", dir_path)

In [ ]:
def prepare_data(exp_dir):
  files = os.listdir(exp_dir)
  for img_name in files:
    path = os.path.join(exp_dir, img_name)
    input_path_back = os.path.join(path, "input_back")
    if not os.path.exists(input_path_back):
        os.mkdir(input_path_back)
    input_path_obj = os.path.join(path, "input_obj")
    if not os.path.exists(input_path_obj):
        os.mkdir(input_path_obj)
    img_path = os.path.join(img_dir, img_name)
    shutil.copy(img_dir+"/"+img_name+"/"+img_name+"_lr.png", input_path_back+"/"+img_name+"_lr_back.png")
    shutil.copy(img_dir+"/"+img_name+"/"+img_name+"_lr.png", input_path_obj+"/"+img_name+"_lr_obj.png")


# run the code

In [ ]:
def Run_ZSSR(input_dir, output_dir, masks_dir):
   !python "/content/drive/MyDrive/KernelGAN-Masks/KernelGAN/train.py" --input-dir $input_dir --output-dir $output_dir --weights-dir $weights_dir --masks-dir $masks_dir --SR


In [ ]:
def save_kernels(back, obj, saving_path):
  plt.gray()
  fig = plt.figure(figsize=(15, 15))

  fig.add_subplot(1, 2, 1)
  plt.imshow(back)
  plt.title("back")

  fig.add_subplot(1, 2, 2)
  plt.imshow(obj)
  plt.title("obj")

  plt.savefig(saving_path+'/kernels.png')

  plt.close()

In [ ]:
def calc_parts(obj_hr_mask, back_hr_mask, hr_obj, hr_back):
  back = cv2.bitwise_and(back_hr_mask, hr_back)
  obj = cv2.bitwise_and(obj_hr_mask, hr_obj)
  full = cv2.add(obj, back)

  return obj, back, full

In [ ]:
def save(img_name, output_path, parent_path, run):
  obj_hr_mask = cv2.imread(os.path.join(masks_dir, img_name+"/obj_hr_mask.png"))
  back_hr_mask = cv2.imread(os.path.join(masks_dir,  img_name+"/back_hr_mask.png"))
  hr_obj =  cv2.imread(os.path.join(output_path,  img_name+"_lr_obj/ZSSR_"+img_name+"_lr_obj.png"))
  hr_back = cv2.imread(os.path.join(output_path,  img_name+"_lr_back/ZSSR_"+img_name+"_lr_back.png"))
  k_obj = scipy.io.loadmat(output_path+'/'+img_name+'_lr_obj/'+img_name+'_lr_obj_kernel_x2.mat')
  k_back = scipy.io.loadmat(output_path+'/'+img_name+'_lr_back/'+img_name+'_lr_back_kernel_x2.mat')
  k_obj = np.array(k_obj.get("Kernel"))
  k_back = np.array(k_back.get("Kernel"))


  obj, back, full = calc_parts(obj_hr_mask, back_hr_mask, hr_obj, hr_back)
  new_out = parent_path+"/out"+str(run)
  if not os.path.exists(new_out):
    os.mkdir(new_out)

  save_kernels(k_back, k_obj, new_out)
  cv2.imwrite(new_out+"/obj.png" , obj)
  cv2.imwrite(new_out+"/back.png" , back)
  cv2.imwrite(new_out+"/full.png" , full)



In [ ]:
def start(exp_dir, masks_dir, lst):
  # files = os.listdir(exp_dir)
  # for img_name in files:
  for img_name in lst:
    for run in range(3):
      print(img_name+" run "+str(run))
      path = os.path.join(exp_dir, img_name)
      input_path_back = os.path.join(path, "input_back")
      input_path_obj = os.path.join(path, "input_obj")

      output_path = os.path.join(path, "output"+str(run))
      if not os.path.exists(output_path):
          os.mkdir(output_path)
      # else:
      #   continue

      Run_ZSSR(input_path_back, output_path, masks_dir)

      Run_ZSSR(input_path_obj, output_path, masks_dir)
      save(img_name, output_path, path, run)


In [ ]:
create_folders(exp_dir)

img136
/content/drive/MyDrive/KernelGAN-Masks/exps/img136
img268
/content/drive/MyDrive/KernelGAN-Masks/exps/img268
img494
/content/drive/MyDrive/KernelGAN-Masks/exps/img494


In [ ]:
image_list = [img for img in os.listdir(img_dir)]
prepare_data(exp_dir)

In [ ]:
start(exp_dir, masks_dir, image_list)

img136 run 0
2024-10-14 11:27:38.631610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 11:27:38.651362: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 11:27:38.657832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Instructions for updating:
non-resource variables are not supported in the long term
Scale Factor: X2 	ZSSR: True 	Real Image: False
************************************************************
STARTED KernelGAN on: "/content/drive/MyDrive/KernelGAN-Masks/exps/img136/input_back/img136_lr_back.png"...

mask_path =  /content/drive/MyDrive/KernelGAN-Mas